In [ ]:
#there's lots of options for models including agnostic but we'll start with NN

#https://shap.readthedocs.io/en/latest/


#SHAPE MAKES THE COEFFICIENT CORRELATION evident
#causal when feature analyzing is strongly independent from others (i.e. why we matched + hope get unobserved)


"""
Look into double ML (CausalML)
- will fail to capture indirect casual effects if control for downstream features caused by feature of interest
"""

"""
Unfortunately, we often don’t know the true causal graph so it can be hard to know when another feature is redundant with our feature of interest because of observed confounding vs. non-confounding redundancy. If it is because of confounding then we should control for that feature using a method like double ML, whereas if it is a downstream consequence then we should drop the feature from our model if we want full causal effects rather than only direct effects. Controlling for a feature we shouldn’t tends to hide or split up causal effects, while failing to control for a feature we should have controlled for tends to infer causal effects that do not exist. This generally makes controlling for a feature the safer option when you are uncertain.

"""
#controlling for feature is safer; confounding -> control, downstream conseqeunce -> drop
#this is actually good for us to know. Our base data is all downstream which could be a problem (meaning we should also try dropping that info from dataset)
#this means we should run it both with and without base data

In [8]:
old_path = "../../Dataset Expansion/raw_data/redditor_dict.json"
import json 

with open(old_path, 'r') as f:
    json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)